In [1]:
# don't ever remove
%env QT_QPA_PLATFORM=wayland7

env: QT_QPA_PLATFORM=wayland7


In [2]:
import wandb
import tensorflow as tf

In [3]:
import random
import gym
import gym_snake
from time import sleep
import numpy as np
from matplotlib import pyplot as plt
import cv2
from gym_snake.envs.constants import Direction4, Action4

register(
    id='Snake--4x4-DeadApple--v0',
    entry_point='gym_snake.envs:Snake__4x4_DeadApple_'
)
register(
    id='Snake-Hex-4x4-DeadApple--v0',
    entry_point='gym_snake.envs:Snake_Hex_4x4_DeadApple_'
)
register(
    id='Snake--8x8-DeadApple--v0',
    entry_point='gym_snake.envs:Snake__8x8_DeadApple_'
)
register(
    id='Snake-Hex-8x8-DeadApple--v0',
    entry_point='gym_snake.envs:Snake_Hex_8x8_DeadApple_'
)
register(
    id='Snake--16x16-DeadApple--v0',
    entry_point='gym_snake.envs:Snake__16x16_DeadApple_'
)
register(
    id='Snake-Hex-16x16-DeadApple--v0',
    entry_point='gym_snake.envs:Snake_Hex_16x16_DeadApple_'
)
register(
    id='Snake--4x4---v0',
    entry_point='gym_snake.envs:Snake__4x4__'
)
register(
    id='Snake-Hex-4x4---v0',
    entry_point='gym_snake.envs:Snake_Hex_4x4__'
)
register(
    id='Snake--8x8---v0',
    entry_point='gym_snake.envs:Snake__8x8__'
)
register(
    id='Snake-Hex-8x8---v0',
    entry_point='gym_snake.envs:Snake_Hex_8x8__'
)
register(
    

In [4]:
%matplotlib tk

In [5]:
env = gym.make("Snake-8x8-v0")

/home/anetakahleova/miniconda3/envs/Snake/lib/python3.9/site-packages/gym/utils/seeding.py:47: DeprecationWarning: WARN: Function `rng.randint(low, [high, size, dtype])` is marked as deprecated and will be removed in the future. Please use `rng.integers(low, [high, size, dtype])` instead.
  deprecation(


In [6]:
obs = env.reset()

In [7]:
obs

array([[[  0,   0,   0],
        [  0,   0,   0],
        [  0,   0,   0],
        [  0,   0,   0],
        [  0,   0, 255],
        [  0,   0,   0],
        [  0,   0,   0],
        [  0,   0,   0]],

       [[  0,   0,   0],
        [  0,   0,   0],
        [  0,   0,   0],
        [  0,   0,   0],
        [  0, 255,   0],
        [  0,   0,   0],
        [  0,   0,   0],
        [  0,   0,   0]],

       [[  0,   0,   0],
        [  0,   0,   0],
        [  0,   0,   0],
        [  0,   0,   0],
        [  0, 255,   0],
        [  0,   0,   0],
        [255,   0,   0],
        [  0,   0,   0]],

       [[  0,   0,   0],
        [  0,   0,   0],
        [  0,   0,   0],
        [  0,   0,   0],
        [  0, 255,   0],
        [  0,   0,   0],
        [  0,   0,   0],
        [  0,   0,   0]],

       [[  0,   0,   0],
        [  0,   0,   0],
        [  0,   0,   0],
        [  0,   0,   0],
        [  0, 255,   0],
        [  0,   0,   0],
        [  0,   0,   0],
        [  0,   0

In [8]:
plt.ion()
plt.show()

def render(obs):
    plt.imshow(obs)
    plt.draw()
    plt.pause(1)


In [9]:
def getlookdir(obs): # funguje jen na prvni frame
    for y in range(obs.shape[1]):
        for x in range(obs.shape[0]):
            pixel = obs[y][x]
            if pixel[2] == 255:
                hlava = [y, x]
            if pixel[1] == 255:
                telo = [y, x]

    if hlava[0] == telo[0]:#stejny radek
        if hlava[1] > telo[1]:
            look_dir = Direction4.east
        else:
            look_dir = Direction4.west
    else:
        if hlava[0] > telo[0]:
            look_dir = Direction4.south
        else:
            look_dir = Direction4.north

    return  look_dir

In [10]:
obs = env.reset()
look_dir = getlookdir(obs)
print(look_dir, end="      ")
for i in range(1000):
    #env.render

    k = input()

    action = random.choice([0])

    if k == "D":
        action = 1
    elif k == "A":
        action = 2
    elif k == "Q":
        break;


    obs, reward, done, info = env.step(action)
    look_dir = look_dir.add_action(action)
    # print(info)

    print(action, look_dir)

    if done:
        obs = env.reset()
        look_dir = getlookdir(obs)
        print(look_dir, end="    ")


    render(obs)
    sleep(0.1)


env.close()

Direction4.west      0 Direction4.west
Direction4.east    0 Direction4.east
0 Direction4.east
0 Direction4.east
0 Direction4.east
Direction4.south    0 Direction4.south
Direction4.east    0 Direction4.east
1 Direction4.south
Direction4.west    0 Direction4.west
0 Direction4.west
0 Direction4.west
0 Direction4.west
Direction4.south    0 Direction4.south
0 Direction4.south
Direction4.east    0 Direction4.east
0 Direction4.east
Direction4.south    0 Direction4.south


# Using Deep Q neural Network on CartPole environment
# https://www.youtube.com/watch?v=dpBKz1wxE_c

In [11]:


from collections import deque
from time import sleep
import gym
import random
import numpy as np
import tensorflow as tf
from tensorflow.python.keras.layers import Dense
from tensorflow.python.keras.models import Sequential
from tensorflow.python.keras.optimizer_v2.adam import Adam

In [12]:
# To fix tf2 speed - https://github.com/tensorflow/tensorflow/issues/33024
tf.compat.v1.disable_eager_execution()


In [13]:
### Prepare algorithm ###
# Q Network - we use one layer so it is not deep
class DQNAgent:
    def __init__(self, env, gamma=0.95, epsilon=1.0, epsilon_decay=0.995, min_epsilon=0.01, learning_rate=0.001,
                 state_dim=None, action_size=None):
        self.gamma = gamma  # discount rate
        self.epsilon = epsilon
        self.epsilon_decay = epsilon_decay
        self.min_epsilon = min_epsilon
        self.learning_rate = learning_rate

        self.state_dim = state_dim or env.observation_space.shape
        self.action_size = action_size or env.action_space.n

        self.replay_buffer = deque(maxlen=1000)
        self.model = self.build_model()

    def build_model(self):
        model = Sequential([
            Dense(128, input_shape=self.state_dim, activation="relu"),
            Dense(128, activation="relu"),
            Dense(128, activation="relu"),
            Dense(self.action_size, activation=None)
        ])
        model.compile(loss="mean_squared_error", optimizer=Adam(lr=self.learning_rate))
        model.summary(line_length=120)
        return model

    def get_action(self, state, skip_random=False):
        if not skip_random and random.random() < self.epsilon:
            action_random = random.choice(range(self.action_size))  # Get random action
            return action_random

        else:
            q_state = self.model.predict(np.array([state]))  # Select Q values for current state from Q table
            action_greedy = np.argmax(q_state)               # Select action with highest Q value
            return action_greedy

    def add_experience(self, experience):
        # print(experience)
        self.replay_buffer.append(experience)

    def train_network(self, batch_size=500):
        # Do not train if not enough experience
        if len(self.replay_buffer) < batch_size:
            return

        samples = random.choices(self.replay_buffer, k=batch_size)
        state, action, next_state, reward, done = (list(col) for col in zip(*samples))  # Make list of states, list of actions, ...

        q_current = self.model.predict(np.array(state))            # Get Q values for all possible current actions

        q_next = self.model.predict(np.array(next_state))          # Get Q values for all possible next actions
        q_next[done] = 0                                                   # Set all future(next) Q values to 0 if this step is the last one
        q_target = reward + (self.gamma * np.max(q_next, axis=1))  # Calculate target Q = reward + max next Q value

        for i in range(batch_size):
            q_update = q_target[i] - q_current[i][action[i]]        # = how much to move Q value in table from current value
            q_current[i][action[i]] += self.learning_rate * q_update   # Update Q value a little based on LEARNING_RATE

        # Train the neural network with our updated Q value
        self.model.fit(np.array(state), np.array(q_current), verbose=0)

In [14]:
def onehot_encode(index, length):
    a = np.zeros((length,), dtype=np.uint8)
    a[index] = 1
    return a

def preprocess_state(obs: np.ndarray, look_dir):                       # obs = next_state
    # next_state = next_state.flatten()                                # odstrani se zavorky... rozbali se numpy array
    # next_state = next_state/255
    # look_dir_onehot = onehot_encode(look_dir, 4)
    # next_state = np.concatenate((next_state, look_dir_onehot))
    obs = np.rot90(obs, look_dir)                                     # otoceni plochy tak, aby se had dival nahoru
    output = np.zeros(7, dtype=np.uint8)


    for y in range(obs.shape[1]):
        for x in range(obs.shape[0]):
            pixel = obs[y][x]
            if pixel[2] == 255:
                hlava = [y, x]
            if pixel[0] == 255:
                jidlo = [y, x]


    output[0] = 1 if hlava[0] > jidlo[0] else 0                         #jidlo nad hlavou
    output[1] = 1 if hlava[0] < jidlo[0] else 0                         #jidlo pod hlavou
    output[2] = 1 if hlava[1] > jidlo[1] else 0                         #jidlo nalevo od hlavy
    output[3] = 1 if hlava[1] < jidlo[1] else 0                         #jidlo napravo od hlavy

    if hlava[0] == 0:
        output[4] = 1
    elif obs[hlava[0]-1][hlava[1]][1]:
        output[4] = 1

    if hlava[1] == 0:
        output[5] = 1
    elif obs[hlava[0]][hlava[1]-1][1]:
        output[5] = 1

    if hlava[1] == obs.shape[1]-1:
        output[6] = 1
    elif obs[hlava[0]][hlava[1]+1][1]:
        output[6] = 1

    return output

In [15]:
### Train ###
agent = DQNAgent(env, learning_rate=0.001, state_dim=(7,))

# Was stable after about 100 episodes
EPISODES = 2000
for episode in range(EPISODES):
    state = env.reset()
    look_dir = getlookdir(state)
    state = preprocess_state(state, look_dir)
    done = False
    cumulative_reward = 0
    i = 0
    env_i = 0

    while not done:
        action = agent.get_action(state)                                 # Action = get action from Agent
        next_state, reward, done, info = env.step(action)                # Do the Action
        orig_state = next_state
        look_dir = look_dir.add_action(action)
        if not done:
            next_state = preprocess_state(next_state, look_dir)
        else:
            next_state = np.zeros(7)

        if done:
            reward = -10  # Punish for crashing

        cumulative_reward += reward

        agent.add_experience([state, action, next_state, reward, done])  # Add experience to replay buffer
        agent.train_network()                                            # Train the network

        i += 1
        state = next_state                                               # Set State to next State

        # if episode % 250 == 0:
        #     render(orig_state)
        # env.render()
        # sleep(0.05)

    # Decay epsilon each episode
    agent.epsilon = max(agent.min_epsilon, agent.epsilon * agent.epsilon_decay)

    print(f"e: {episode:03}, eps: {agent.epsilon:.2f}, r: {cumulative_reward}")

/home/anetakahleova/miniconda3/envs/Snake/lib/python3.9/site-packages/tensorflow/python/keras/optimizer_v2/optimizer_v2.py:368: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  warnings.warn(


Model: "sequential"
________________________________________________________________________________________________________________________
Layer (type)                                          Output Shape                                    Param #           
dense (Dense)                                         (None, 128)                                     1024              
________________________________________________________________________________________________________________________
dense_1 (Dense)                                       (None, 128)                                     16512             
________________________________________________________________________________________________________________________
dense_2 (Dense)                                       (None, 128)                                     16512             
________________________________________________________________________________________________________________________
dense_3 (Den

/home/anetakahleova/miniconda3/envs/Snake/lib/python3.9/site-packages/tensorflow/python/keras/engine/training.py:2458: UserWarning: `Model.state_updates` will be removed in a future version. This property should not be used in TensorFlow 2.0, as `updates` are applied automatically.
  warnings.warn('`Model.state_updates` will be removed in a future version. '
2022-07-31 03:28:22.783225: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2 AVX AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


e: 008, eps: 0.96, r: -10
e: 009, eps: 0.95, r: -10
e: 010, eps: 0.95, r: -10
e: 011, eps: 0.94, r: -10
e: 012, eps: 0.94, r: -10
e: 013, eps: 0.93, r: -10
e: 014, eps: 0.93, r: -10
e: 015, eps: 0.92, r: -10
e: 016, eps: 0.92, r: -10
e: 017, eps: 0.91, r: -10
e: 018, eps: 0.91, r: -10
e: 019, eps: 0.90, r: -10
e: 020, eps: 0.90, r: -10
e: 021, eps: 0.90, r: -10
e: 022, eps: 0.89, r: -10
e: 023, eps: 0.89, r: -10
e: 024, eps: 0.88, r: -10
e: 025, eps: 0.88, r: -10
e: 026, eps: 0.87, r: -10
e: 027, eps: 0.87, r: -10
e: 028, eps: 0.86, r: -10
e: 029, eps: 0.86, r: -9
e: 030, eps: 0.86, r: -10
e: 031, eps: 0.85, r: -10
e: 032, eps: 0.85, r: -10
e: 033, eps: 0.84, r: -10
e: 034, eps: 0.84, r: -10
e: 035, eps: 0.83, r: -10
e: 036, eps: 0.83, r: -10
e: 037, eps: 0.83, r: -10
e: 038, eps: 0.82, r: -10
e: 039, eps: 0.82, r: -10
e: 040, eps: 0.81, r: -10
e: 041, eps: 0.81, r: -10
e: 042, eps: 0.81, r: -10
e: 043, eps: 0.80, r: -10
e: 044, eps: 0.80, r: -10
e: 045, eps: 0.79, r: -10
e: 046, eps: 


KeyboardInterrupt



In [ ]:
### Test run ###
print()
while True:
    obs = env.reset()
    look_dir = getlookdir(obs)
    done = False
    while not done:
        state = preprocess_state(obs, look_dir)
        action = agent.get_action(state, skip_random=True)              # Action = get action from Agent
        obs, reward, done, info = env.step(action)  # Do the Action
        look_dir = look_dir.add_action(action)
        render(obs)
        sleep(0.5)
